## Error Detection Challenge

In [60]:
#importing necessary libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder, LabelBinarizer #for data transformations
%matplotlib inline

In [61]:
os.getcwd()

'C:\\Users\\User\\Desktop\\competitive-programming\\sea_turtle_challenge'

In [62]:
path = 'sea_turtle_challenge/data/'

#### Reading in the data files.
Here i use **latin-1** as my encoding to walk-around the 
'can't decode'unicode error that occurs on usage of the usual 
pd.read_csv('**<'dataset'>.csv**') method. The reason is that the files may not be 
in real csv format but instead html format.
the **cp1252** encoding could as well solve the issue

In [63]:
dirty_data = pd.read_csv('../sea_turtle_challenge/data/dirty_data.csv', encoding='latin-1')
clean_data = pd.read_csv('../sea_turtle_challenge/data/cleaned_data.csv', encoding='latin-1')
test_data = pd.read_csv('../sea_turtle_challenge/data/test_data.csv', encoding='latin-1')

In [64]:
# previewing the first few dirty data records
dirty_data.head()

,Rescue_ID,Date_Caught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,...,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure
0,1998_RE_0001,4/14/1998,researcher_12,site_110,creek,net,fisher_619,site_58,species_1,site_80,...,58.5,NaN,Unknown,NaN,Released,4/14/1998,NaN,NaN,NaN,NaN
1,1998_RE_0002,7/7/1998,researcher_4,not_recorded,ocean,longline,fisher_522,site_22,species_1,not_recorded,...,37.0,NaN,NaN,1B-1LLS,NaN,NaN,NaN,NaN,NaN,NaN
2,1998_RE_0003,8/3/1998,NaN,site_12,creek,net,fisher_1254,not_recorded,species_1,site_109,...,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1998_RE_0004,8/7/1998,researcher_12,site_110,creek,net,fisher_360,not_recorded,species_2,site_113,...,31.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998_RE_0005,9/25/1998,researcher_17,not_recorded,creek,collected floater,fisher_865,site_8,species_3,site_109,...,63.5,NaN,NaN,Bs on C+ old panga wounds,NaN,NaN,Found trapped in mangroves,NaN,NaN,NaN


In [65]:
# previewing the first few cleaned data records
clean_data.head()

,Rescue_ID,Date_Caught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,...,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure
0,1998_RE_0001,4/17/1998,researcher_19,site_110,creek,net,fisher_619,site_58,species_1,site_80,...,58.42,NaN,Unknown,NaN,Released,4/17/1998,NaN,NaN,NaN,NaN
1,1998_RE_0002,7/7/1998,researcher_15,site_15,creek,longline,fisher_522,site_22,species_1,not_recorded,...,36.83,NaN,Unknown,1B-1LLS,NaN,NaN,NaN,NaN,NaN,NaN
2,1998_RE_0003,8/3/1998,not_recorded,site_12,creek,not_recorded,fisher_1254,not_recorded,species_1,site_109,...,33.00,NaN,Unknown,NaN,Released,8/3/1998,NaN,NaN,NaN,NaN
3,1998_RE_0004,8/7/1998,researcher_19,site_110,creek,not_recorded,fisher_360,not_recorded,species_2,site_108,...,31.75,NaN,Unknown,There was pillings on carapace.,Released,8/7/1998,NaN,NaN,NaN,NaN
4,1998_RE_0005,9/25/1998,researcher_17,site_8,creek,collected floater,fisher_865,site_8,species_3,site_121,...,63.50,NaN,Unknown,Bs on C+ old panga wounds,Released,9/25/1998,Found trapped in mangroves,NaN,NaN,NaN


In [66]:
#filling the NaN columns with 1's to indicate no errors
dirty_data.fillna(0, inplace=True)
clean_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

### Target variables
Since the model is going to be trained on both clean and dirty data, we have to generate the target variables *(errors of the respective columns)* by stacking the clean and dirty dataframes side by side

In [67]:
#generating targets values (errors)
targets = dirty_data.where(dirty_data.values==clean_data.values)
targets.head()

,Rescue_ID,Date_Caught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,...,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure
0,1998_RE_0001,NaN,NaN,site_110,creek,net,fisher_619,site_58,species_1,site_80,...,NaN,0.0,Unknown,0,Released,NaN,0,0,0.0,0.0
1,1998_RE_0002,7/7/1998,NaN,NaN,NaN,longline,fisher_522,site_22,species_1,not_recorded,...,NaN,0.0,NaN,1B-1LLS,0,0,0,0,0.0,0.0
2,1998_RE_0003,8/3/1998,NaN,site_12,creek,NaN,fisher_1254,not_recorded,species_1,site_109,...,33.0,0.0,NaN,0,NaN,NaN,0,0,0.0,0.0
3,1998_RE_0004,8/7/1998,NaN,site_110,creek,NaN,fisher_360,not_recorded,species_2,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0,0,0.0,0.0
4,1998_RE_0005,9/25/1998,researcher_17,NaN,creek,collected floater,fisher_865,site_8,species_3,NaN,...,63.5,0.0,NaN,Bs on C+ old panga wounds,NaN,NaN,Found trapped in mangroves,0,0.0,0.0


In [68]:
#filling the new NaN targets to represent 1 (error)
targets.fillna(1, inplace=True)

#dropping the rescue_ID column as it tells nothing about the error
targets.drop('Rescue_ID', axis=1, inplace=True)

*replacing non-1 entries in the targets dataframe as 0 to indicate no error since they match in both the dirty and cleaned data*

In [69]:
targets = targets.replace(dirty_data.where(dirty_data.values==clean_data.values), 0)

In [70]:
targets.head()

,Date_Caught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,Tag_1,...,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure
0,1,1,0,0,0,0,0,0,0,1,...,1.0,0.0,0,0,0,1,0,0,0.0,0.0
1,0,1,1,1,0,0,0,0,0,0,...,1.0,0.0,1,0,0,0,0,0,0.0,0.0
2,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,1,0,1,1,0,0,0.0,0.0
3,0,1,0,0,1,0,0,0,1,0,...,1.0,0.0,1,1,1,1,0,0,0.0,0.0
4,0,0,1,0,0,0,0,0,1,0,...,0.0,0.0,1,0,1,1,0,0,0.0,0.0


In [71]:
dirty_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4292 entries, 0 to 4291
Data columns (total 26 columns):
Rescue_ID                4292 non-null object
Date_Caught              4292 non-null object
Researcher               4292 non-null object
CaptureSite              4292 non-null object
ForagingGround           4292 non-null object
CaptureMethod            4292 non-null object
Fisher                   4292 non-null object
LandingSite              4292 non-null object
Species                  4292 non-null object
ReleaseSite              4292 non-null object
Tag_1                    4292 non-null object
Tag_2                    4292 non-null object
Tag_3                    4292 non-null float64
Lost_Tags                4292 non-null object
T_Number                 4292 non-null object
CCL_cm                   4292 non-null float64
CCW_cm                   4292 non-null float64
Weight_Kg                4292 non-null float64
Sex                      4292 non-null object
TurtleCharacter

In [72]:
dirty_data["Date_Caught"] = pd.to_datetime(dirty_data.Date_Caught)
test_data["Date_Caught"] = pd.to_datetime(test_data.Date_Caught)

In [73]:
dirty_data["year"] = dirty_data["Date_Caught"].dt.year
test_data["year"] = test_data["Date_Caught"].dt.year

In [74]:
#dropping the Id columns in the train and test data
test_data.drop(["Rescue_ID", "Date_Caught"], axis=1, inplace=True)
dirty_data.drop(["Rescue_ID", "Date_Caught"], axis=1, inplace=True)

#training features and labels
features = dirty_data
labels = targets

In [75]:
features.head()

,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,Tag_1,Tag_2,...,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure,year
0,researcher_12,site_110,creek,net,fisher_619,site_58,species_1,site_80,Missing data,None,...,0.0,Unknown,0,Released,4/14/1998,0,0,0.0,0.0,1998
1,researcher_4,not_recorded,ocean,longline,fisher_522,site_22,species_1,not_recorded,NotTagged_0002,None,...,0.0,0,1B-1LLS,0,0,0,0,0.0,0.0,1998
2,0,site_12,creek,net,fisher_1254,not_recorded,species_1,site_109,NotTagged_0003,None,...,0.0,0,0,0,0,0,0,0.0,0.0,1998
3,researcher_12,site_110,creek,net,fisher_360,not_recorded,species_2,site_113,NotTagged_0004,None,...,0.0,0,0,0,0,0,0,0.0,0.0,1998
4,researcher_17,not_recorded,creek,collected floater,fisher_865,site_8,species_3,site_109,NotTagged_0005,None,...,0.0,0,Bs on C+ old panga wounds,0,0,Found trapped in mangroves,0,0.0,0.0,1998


In [86]:
labels.head()

,Date_Caught,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,Tag_1,...,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure
0,1,1,0,0,0,0,0,0,0,1,...,1.0,0.0,0,0,0,1,0,0,0.0,0.0
1,0,1,1,1,0,0,0,0,0,0,...,1.0,0.0,1,0,0,0,0,0,0.0,0.0
2,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,1,0,1,1,0,0,0.0,0.0
3,0,1,0,0,1,0,0,0,1,0,...,1.0,0.0,1,1,1,1,0,0,0.0,0.0
4,0,0,1,0,0,0,0,0,1,0,...,0.0,0.0,1,0,1,1,0,0,0.0,0.0


In [76]:
""" applying transformations to the dirty and test data i.e. text to int conversion """
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
"""learning vocabulary of the training data"""
vect.fit(features)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [77]:
#examining fitted vocabulary
vect.get_feature_names()

['capturemethod',
 'capturesite',
 'ccl_cm',
 'ccw_cm',
 'date_release',
 'expenditure',
 'fisher',
 'foragingground',
 'landingsite',
 'lost_tags',
 'pcvnumber',
 'release_admiss_notes',
 'releasesite',
 'researcher',
 'sex',
 'specialremarks',
 'species',
 'status',
 't_number',
 'tag_1',
 'tag_2',
 'tag_3',
 'turtlecharacteristics',
 'weight_kg',
 'year']

In [78]:
#transforming training data into a document-term matrix
feats = vect.transform(features)
feats

<25x25 sparse matrix of type '<class 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [79]:
#converting the sparse-matrix to a dense-matrix
feats.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
      

In [80]:
#examining the vocabulary and document-term matrix together
new_features = pd.DataFrame(feats.toarray(), columns = vect.get_feature_names())
new_features.head()

,capturemethod,capturesite,ccl_cm,ccw_cm,date_release,expenditure,fisher,foragingground,landingsite,lost_tags,...,specialremarks,species,status,t_number,tag_1,tag_2,tag_3,turtlecharacteristics,weight_kg,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Training the model

In [81]:
#importing the model and scoring metrics from the sklearn library
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [82]:
model = RandomForestClassifier(n_estimators=100, criterion='mae', n_jobs=-1, random_state=2)

In [83]:
model.fit(new_features, labels)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\User\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\User\Anaconda3\lib\runpy.py i

In [47]:
""" reading the sample submission file and checking its shape """

sample_sub = pd.read_csv('submission_example.csv')
sample_sub.shape

(34050, 2)

In [49]:
sample_sub.head()

,ID,error
0,2011_RE_0001 x Date_Caught,0.0
1,2011_RE_0001 x Researcher,1.0
2,2011_RE_0001 x CaptureSite,1.0
3,2011_RE_0001 x ForagingGround,0.0
4,2011_RE_0001 x CaptureMethod,1.0


In [48]:
""" checking the shape of our test data """

test_data.shape

(1362, 26)

In [40]:
test_data.head()

,Researcher,CaptureSite,ForagingGround,CaptureMethod,Fisher,LandingSite,Species,ReleaseSite,Tag_1,Tag_2,...,CCW_cm,Weight_Kg,Sex,TurtleCharacteristics,Status,Date_Release,Release_Admiss_Notes,SpecialRemarks,PCVNumber,Expenditure
0,researcher_4,site_38,ocean,net,fisher_360,site_118,species_1,site_80,KEB7543,KEB7544,...,92.4,0.0,0.0,Looks healthy& well fed. Small barnacles on bo...,0,0,0,0,0.0,0.0
1,researcher_4,site_36,creek,longline,fisher_1118,site_58,species_1,site_80,KE1970,None,...,53.7,23.5,0.0,Green algae on the carapce. A small hole of 1c...,0,0,0,0,0.0,0.0
2,researcher_4,site_53,creek,net,fisher_703,site_8,species_1,site_80,KE5881,None,...,53.6,25.0,0.0,A V-notch between 1st and 2nd inner scales of ...,0,0,0,0,0.0,0.0
3,researcher_4,site_53,creek,net,fisher_861,site_8,species_1,site_80,KE7514,None,...,53.9,25.5,0.0,Thin green algae on the carapace. Clean plastr...,0,0,0,0,0.0,0.0
4,researcher_4,site_53,creek,net,fisher_242,site_8,species_1,site_80,KE5972,None,...,48.5,20.0,0.0,Flaking on the shell. Additional small CS betw...,0,0,caught a day before,0,0.0,0.0
